# OPERATIONALIZE FRAMEWORK

## 1. Imports

In [1]:
from files_to_database import main_to_database

from operator import itemgetter

import chromadb
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.storage import create_kv_docstore, LocalFileStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.messages import get_buffer_string
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate, format_document, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from getpass import getpass
import os
import pandas as pd
from langchain_core.runnables import RunnableParallel
from langchain.retrievers import ParentDocumentRetriever
from tqdm import tqdm

from transformers import pipeline

## 2. Embedding models (local)

In [2]:
# # For Apple Silicon users: run the following code to make use of MPS (Apple's Metal Performance Shaders) for faster computation
# import torch
# 
# # set device to MPS
# device = torch.device("mps")
# 
# # empty cache and set memory fraction
# torch.mps.empty_cache()
# torch.mps.set_per_process_memory_fraction(0.9)
# 
# # choose embeddings model
# multilingual = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# 
# # local embedding model, download to cache folder
# embedding_model = SentenceTransformer(multilingual, cache_folder="../Data/sentence_transformers", device=device)
# 
# embeddings_retrieve = HuggingFaceEmbeddings(model_name=multilingual, cache_folder="../Data/sentence_transformers")
# 
# # move model to MPS
# embedding_model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [3]:
# local embedding model, download to cache folder
multilingual = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# model used for document embedding
embedding_model = SentenceTransformer(
    model_name_or_path=multilingual, 
    cache_folder="../../Data/sentence_transformers"
)

In [4]:
# model used for query embedding
embeddings_retrieve = HuggingFaceEmbeddings(
    model_name=multilingual,
    cache_folder="../../Data/sentence_transformers"
)

## 3. Chroma client setup

In [5]:
# initiate the chroma client, which is the interface to the database
database_path = "../Data/my_vectordb"
chroma_client = chromadb.PersistentClient(path=database_path)

In [6]:
# print the collections
chroma_client.list_collections()

[Collection(name=tenderned),
 Collection(name=binnenlands_bestuur),
 Collection(name=rijksoverheid),
 Collection(name=ibestuur)]

## 4. Retrievers for all databases

4.1 Parent / child splitters

In [7]:
# define the retrievers, parent and child splitters, MAKE SURE TO CHANGE ALSO IN files_to_database.py
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                 chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=128,
                                                chunk_overlap=0)

4.2 Rijksoverheid retriever

In [8]:
# rijksoverheid database
rijksoverheid_db= Chroma(
    collection_name="rijksoverheid",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [9]:
columns_to_embed = ["content"]
columns_to_metadata = ["id", "type", "title", "canonical", "introduction", "lastmodified", "available", "initialdate"]

# rijksoverheid retriever
full_path = os.path.abspath("../Data/my_vectordb/full_documents/rijksoverheid")

fs = LocalFileStore(full_path)
store = create_kv_docstore(fs)

rijksoverheid_db_retriever = ParentDocumentRetriever(
    vectorstore=rijksoverheid_db,
    docstore=store,
    child_splitter=child_splitter,
    child_metadata_field=columns_to_metadata,
    parent_splitter=parent_splitter,
    )

4.3 ibestuur retriever

In [10]:
# ibestuur database
ibestuur_db= Chroma(
    collection_name="ibestuur",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [11]:
# ibestuur retriever
ibestuur_retriever = ibestuur_db.as_retriever(
    search_kwargs={"k": 1}
)

4.4 binnenlandsbestuur retriever

In [12]:
# binnenlandsbestuur database
binnenlandsbestuur_db= Chroma(
    collection_name="binnenlands_bestuur",
    client=chroma_client,
    persist_directory="../Data/my_vectordb",
    embedding_function=embeddings_retrieve,
)

In [13]:
# binnenlandsbestuur retriever
binnenlandsbestuur_retriever = binnenlandsbestuur_db.as_retriever(
    search_kwargs={"k": 1}
)

4.5 tenderned retriever

In [14]:
# tenderned database

## 5. LLM from Inference Endpoints API

In [15]:
# Make sure to replace these values with your personal API URL and KEY
# API_URL = "https://oi6h8u843v8nt5qt.eu-west-1.aws.endpoints.huggingface.cloud"
# API_KEY = getpass("Enter your API KEY:")

In [16]:
# LLM model (Hugging Face)
# llm = HuggingFaceEndpoint(
#     endpoint_url=API_URL,
#     huggingfacehub_api_token=API_KEY,
#     temperature=0.1,
#     max_new_tokens=2048,
#     model_kwargs={"max_input_length": 2048, "max_length": 2048, "max_num_tokens": 2048}
# )

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/csnoeij/.cache/huggingface/token
Login successful


In [ ]:
HF_token = getpass("Enter your Hugging Face API Token:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_token
llm = HuggingFaceHub(
    huggingfacehub_api_token=HF_token,
    repo_id="HuggingFaceH4/zephyr-7b-alpha",
    model_kwargs={"temperature":0.5, "max_new_tokens":512, "max_length":64}
)
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 128}
)

## 6. Define functions to operationalize framework

6.1 Generate prompt

In [17]:
# chat prompt template
template = """
Je bent 'GovTech-GPT', een geavanceerde AI-assistent met uitgebreide expertise in digitale technologieën specifiek gericht op toepassingen binnen de Nederlandse overheid. Je belangrijkste taak is het ondersteunen bij het operationaliseren van e-gov benchmarking frameworks. Je antwoordt altijd op basis van de meest recente gegevens en inzichten, en houdt rekening met de specifieke context van de Nederlandse overheid. Antwoorden geef je alleen volgens het gespecificeerde dataformat, waarbij je, indien mogelijk, het cijfer gebruikt en niet de tekst. Voeg verder geen enkele tekst, toelichting of uitleg meer toe. Als je het antwoord niet weet, geef je geen fictieve informatie of uitleg, maar antwoord enkel en alleen met: 'Geen antwoord.'  \n\n"

CONTEXT: {context}

DATA FORMAT: {data_format}

VRAAG: {question}

ANTWOORD: 
"""

In [18]:
# chat prompt template
prompt = ChatPromptTemplate.from_template(template)

In [19]:
# output parser
output_parser = StrOutputParser()

6.2 Format context

In [20]:
# format the context for input
def format_context(context): 
    context_string = ""
    
    for i in range(len(context["context_ibestuur"])):
        context_string += f"{dict(context['context_ibestuur'][i])['page_content']}\n"
    for i in range(len(context["context_rijksoverheid"])):
        context_string += f"{dict(context['context_rijksoverheid'][i])['page_content']}\n"

    return context_string

6.3 Retrieval setup

In [21]:
# output parser
retrieval = RunnableParallel(
    {
        "context_ibestuur": ibestuur_retriever, 
        "context_rijksoverheid": rijksoverheid_db_retriever, 
        "context_binnenlandsbestuur": binnenlandsbestuur_retriever,
        "question": RunnablePassthrough()
    }
)

6.4 Chain setup

In [ ]:
# function to return docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [22]:
# Chain to generate answer
chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_context(x["context"])))
        | prompt 
        | llm 
        | output_parser
)

# Chain to include used sources + answer
rag_chain_with_source = RunnableParallel(
    {"context": retrieval, 
     "question": RunnablePassthrough(), 
     "data_format": RunnablePassthrough()}
).assign(answer=chain_from_docs)

## 7. Load Framework

In [23]:
# prevent reading extra unnamed column
framework = pd.read_csv("../Results/framework_questions_translated.csv", usecols=[' #', '2022 GTMI Indicators & Sub-indicators NL', 'Response options & Data format NL'])

In [24]:
framework = framework.head()

## 8. Operationalize framework functions

In [25]:
def get_main_indicator(index):
    if '.' in index:
        return index.split('.')[0]
    return index

In [26]:
def generate_question(indicator, sub_indicator):
	if sub_indicator:
		question = f"{indicator}, indien ja, wat is de {sub_indicator} ?"
	else:
		question = f"{indicator} ?"
	return question

In [31]:
def operationalize_framework(framework):
    framework_operationalized = framework.copy()
    framework_operationalized["Operationalisatie"] = None
    framework_operationalized['Prompt'] = None
    framework_operationalized[' #'] = framework_operationalized[' #'].str.replace('I-', '')

    for index, row in tqdm(framework_operationalized.iterrows(), total=framework_operationalized.shape[0]):
        idx = row[' #']
        main_indicator_idx = get_main_indicator(idx)
        sub_indicator_idx = idx.split('.')[1] if '.' in idx else None
        indicator_info = row['2022 GTMI Indicators & Sub-indicators NL']
        data_format = row['Response options & Data format NL']
                
        if sub_indicator_idx:
            # look up the main indicator
            main_indicator = framework_operationalized.loc[framework_operationalized[' #'] == main_indicator_idx]['2022 GTMI Indicators & Sub-indicators NL'].iloc[0]
            question = generate_question(main_indicator, indicator_info)
        else:
            question = generate_question(indicator_info, None)    
        
        output = rag_chain_with_source.invoke(question, {"data_format": data_format})
        
        framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Operationalisatie'] = output
        framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Operationalisatie'] = output["answer"].split("Answer: ")[-1].strip()
        framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Prompt'] = output["answer"].split("Answer: ")[0].strip()
        framework_operationalized.loc[framework_operationalized[' #'] == idx, 'Context'] = output["context"]
        
    return framework_operationalized

## 9. RUN THIS AWeSOME OPERATiONALIZER

In [32]:
from langchain.globals import set_debug

set_debug(True)

In [33]:
df = operationalize_framework(framework)

  0%|          | 0/5 [00:00<?, ?it/s]

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Is er een gedeeld cloud platform beschikbaar voor alle overheids organisaties? ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,data_format>] Entering Chain run with input:
{
  "input": "Is er een gedeeld cloud platform beschikbaar voor alle overheids organisaties? ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,data_format> > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Is er een gedeeld cloud platform beschikbaar voor alle overheids organisaties? ?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,data_format> > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Is er een gedeeld cloud platform beschikbaar voor alle overheids organisaties? ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question,dat

 20%|██        | 1/5 [00:01<00:04,  1.08s/it]

[llm/end] [1:chain:RunnableSequence > 9:chain:RunnableAssign<answer> > 10:chain:RunnableParallel<answer> > 11:chain:RunnableSequence > 16:llm:HuggingFaceEndpoint] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Geen antwoord.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 9:chain:RunnableAssign<answer> > 10:chain:RunnableParallel<answer> > 11:chain:RunnableSequence > 17:parser:StrOutputParser] Entering Parser run with input:
{
  "input": "Geen antwoord."
}
[chain/end] [1:chain:RunnableSequence > 9:chain:RunnableAssign<answer> > 10:chain:RunnableParallel<answer> > 11:chain:RunnableSequence > 17:parser:StrOutputParser] s] Exiting Parser run with output:
{
  "output": "Geen antwoord."
}
[chain/end] [1:chain:RunnableSequence > 9:chain:RunnableAssign<answer> > 10:chain:RunnableParallel<answer> > 11:chain:RunnableSequence] s] Exiting Chain ru

 40%|████      | 2/5 [00:01<00:02,  1.17it/s]

...

[llm/end] [1:chain:RunnableSequence > 10:chain:RunnableAssign<answer> > 11:chain:RunnableParallel<answer> > 12:chain:RunnableSequence > 17:llm:HuggingFaceEndpoint] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Geen antwoord.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 10:chain:RunnableAssign<answer> > 11:chain:RunnableParallel<answer> > 12:chain:RunnableSequence > 18:parser:StrOutputParser] Entering Parser run with input:
{
  "input": "Geen antwoord."
}
[chain/end] [1:chain:RunnableSequence > 10:chain:RunnableAssign<answer> > 11:chain:RunnableParallel<answer> > 12:chain:RunnableSequence > 18:parser:StrOutputParser] s] Exiting Parser run with output:
{
  "output": "Geen antwoord."
}
[chain/end] [1:chain:RunnableSequence > 10:chain:RunnableAssign<answer> > 11:chain:RunnableParallel<answer> > 12:chain:RunnableSequence] s] Exiting Chai

 80%|████████  | 4/5 [00:03<00:00,  1.31it/s]

[llm/end] [1:chain:RunnableSequence > 8:chain:RunnableAssign<answer> > 9:chain:RunnableParallel<answer> > 10:chain:RunnableSequence > 15:llm:HuggingFaceEndpoint] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Geen antwoord.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 8:chain:RunnableAssign<answer> > 9:chain:RunnableParallel<answer> > 10:chain:RunnableSequence > 16:parser:StrOutputParser] Entering Parser run with input:
{
  "input": "Geen antwoord."
}
[chain/end] [1:chain:RunnableSequence > 8:chain:RunnableAssign<answer> > 9:chain:RunnableParallel<answer> > 10:chain:RunnableSequence > 16:parser:StrOutputParser] s] Exiting Parser run with output:
{
  "output": "Geen antwoord."
}
[chain/end] [1:chain:RunnableSequence > 8:chain:RunnableAssign<answer> > 9:chain:RunnableParallel<answer> > 10:chain:RunnableSequence] s] Exiting Chain run wi

100%|██████████| 5/5 [00:07<00:00,  1.45s/it]

[llm/end] [1:chain:RunnableSequence > 9:chain:RunnableAssign<answer> > 10:chain:RunnableParallel<answer> > 11:chain:RunnableSequence > 16:llm:HuggingFaceEndpoint] [3.96s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ja, er is een gedeeld cloud platform beschikbaar voor alle overheids organisaties. De Type beschikbaar cloud platform is een hybride cloud platform, waarbij gebruik wordt gemaakt van zowel publieke als private cloudomgevingen om de interoperabiliteit en dataportabiliteit te waarborgen, terwijl ook de beveiliging en anti-vendor lock-in worden aangepakt.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 9:chain:RunnableAssign<answer> > 10:chain:RunnableParallel<answer> > 11:chain:RunnableSequence > 17:parser:StrOutputParser] Entering Parser run with input:
{
  "input": "Ja, er is een gedeeld cloud platform beschikbaar voor alle o

In [29]:
df

,#,2022 GTMI Indicators & Sub-indicators NL,Response options & Data format NL,Operationalisatie,Prompt,Context
0,1,Is er een gedeeld cloud platform beschikbaar v...,"0= Nee, 1= Alleen cloud strategie/beleid (nog ...",Geen antwoord.,Geen antwoord.,NaN
1,1.1,Naam van het Overheids Cloud platform,Tekst,Geen antwoord.,Geen antwoord.,NaN
2,1.2,Cloud platform / strategie URL,URL,Geen antwoord.,Geen antwoord.,NaN
3,1.3,Overheids Cloud gelanceerd / zal worden gelanc...,YYYY,Geen antwoord.,Geen antwoord.,NaN
4,1.4,Type beschikbaar cloud platform,"0= Onbekend, 1= Publiek (Commercieel), 2= Priv...",Er is een gedeeld cloud platform beschikbaar v...,Er is een gedeeld cloud platform beschikbaar v...,NaN


In [30]:
for index, row in df.iterrows():
	print(f"PROMPT: {row['Prompt']}")
	print(f"{row['Operationalisatie']}")
	print("\n")

PROMPT: Geen antwoord.
Geen antwoord.


PROMPT: Geen antwoord.
Geen antwoord.


PROMPT: Geen antwoord.
Geen antwoord.


PROMPT: Geen antwoord.
Geen antwoord.


PROMPT: Er is een gedeeld cloud platform beschikbaar voor alle overheids organisaties. De Type beschikbaar cloud platform is een hybride cloud platform, waarbij gebruik wordt gemaakt van zowel publieke als private cloudomgevingen om de interoperabiliteit, dataportabiliteit en beveiliging te waarborgen. Dit platform is ontworpen om te voldoen aan de specifieke eisen van de Nederlandse overheid, inclusief de naleving van wet- en regelgeving en de behoefte aan een hoge mate van betrouwbaarheid en veiligheid.
Er is een gedeeld cloud platform beschikbaar voor alle overheids organisaties. De Type beschikbaar cloud platform is een hybride cloud platform, waarbij gebruik wordt gemaakt van zowel publieke als private cloudomgevingen om de interoperabiliteit, dataportabiliteit en beveiliging te waarborgen. Dit platform is ontworpen om te v

In [ ]:
# questions list to ask the model
questions_list = [
    "What is the role of the government in the Netherlands?",
    "What is the role of AI in the Netherlands?",
    "How many AI startups are there in the Netherlands?"
]

In [ ]:
# fill the dataframe with answers and context
def fill_framework(questions):
    rows = []
    for question in questions:
        response = rag_chain_with_source.invoke(question)
        row = {
                "Question": question,
                "Context": response["context"],
                "Answer": response["answer"].split("Answer: ")[-1].strip()
            },
        rows.append(row)
        
    dataframe = pd.DataFrame(
        [item for sublist in rows for item in sublist]
    )
    return dataframe

In [ ]:
df = fill_framework(questions_list)

In [ ]:
df

In [ ]:
# print context
print(df["Context"][0])

# NOTES

### test retrieval parent/child

In [10]:
sub_docs = rijksoverheid_db.similarity_search("Informatiebeveiliging")
print("Child Splits:\n\n", sub_docs[0], "\n\n", sub_docs[1])

Child Splits:

 page_content='. In lijn met de Baseline  Informatiebeveiliging Overheid.     4' metadata={'available': '2020-11-18T13:06:00.000Z', 'canonical': 'https://www.rijksoverheid.nl/documenten/kamerstukken/2020/11/18/kamerbrief-over-stelsel-van-basisregistraties', 'doc_id': '1d9faeeb-fa4c-4ce7-9dda-99a43c673583', 'id': 'bcf61f93-c74d-4a0a-b473-0028516af001', 'initialdate': '2020-11-18T14:38:35.826+01:00', 'introduction': "Staatssecretaris Knops informeert de Tweede Kamer over\xa0actuele ontwikkelingen over het verder verbeteren van het stelsel van basisregistraties, bijvoorbeeld de Basisregistratie Personen (voorheen\xa0gemeentelijke basisadministratie voor persoonsgegevens). Kamerbrief over stelsel van basisregistraties (PDF | 11 pagina's | 557 kB)", 'lastmodified': '2021-12-20T14:20:22.770Z', 'title': 'Kamerbrief over stelsel van basisregistraties', 'type': 'kamerstuk'} 

 page_content='.     Het voorgaande is in het bijzonder van belang voor de informatiebeveiliging' metadat

In [13]:
retrieved_docs = rijksoverheid_db_retriever.invoke("Informatiebeveiliging")
print("Parent Splits:\n\n", retrieved_docs[1], "\n\n", retrieved_docs[1])

IndexError: list index out of range

### CHAIN for single retriever test

In [22]:
# define the chain
chain = (
    {'context': rijksoverheid_db_retriever, 'query': RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

NameError: name 'llm' is not defined

In [0]:
# print the chain
chain.get_graph().print_ascii()

In [0]:
query = "What is the definition of Artificial Intelligence"

response = chain.invoke(query)

In [0]:
print(response)

### Template

In [ ]:
# chat prompt template
prompt_str = """Answer the question below using the context:

Context:
{context}

Question: {question}

Answer: """

# chat prompt
prompt = ChatPromptTemplate.from_template(prompt_str)

## Local LLM (for testing)